# Query playground

In [1]:
import os
os.getcwd()

'/Users/will/Documents/github/cartar/alpine-fng.github.io/data'

In [2]:
import pandas as pd
import numpy as np
import sqlite3

from queries import get_race_data, get_point_total, get_races_list, audit_df

In [3]:
# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [4]:
race_list = get_races_list(conn)

In [5]:
race_list.head(3)

,race_id,race_date,description,year
0,240218,2024-02-18 00:00:00,2024 F&G Race #4,2024
1,240211,2024-02-11 00:00:00,2024 F&G Race #3,2024
2,240121,2024-01-21 00:00:00,2024 F&G Race #2,2024


In [ ]:
list[race_list]

In [6]:
years =[2020, 2022, 2023,2024]
races_by_year = {}

In [24]:
races_for_year = race_list[race_list['year'] == '2024']
races_list = races_for_year.to_dict('records')
races_by_year['2024'] = races_list

In [21]:
races_for_year

,race_id,race_date,description,year
0,240218,2024-02-18 00:00:00,2024 F&G Race #4,2024
1,240211,2024-02-11 00:00:00,2024 F&G Race #3,2024
2,240121,2024-01-21 00:00:00,2024 F&G Race #2,2024
3,240114,2024-01-14 00:00:00,2024 F&G Race #1,2024


In [22]:
races_list

[{'race_id': 240218,
  'race_date': '2024-02-18 00:00:00',
  'description': '2024 F&G Race #4',
  'year': '2024'},
 {'race_id': 240211,
  'race_date': '2024-02-11 00:00:00',
  'description': '2024 F&G Race #3',
  'year': '2024'},
 {'race_id': 240121,
  'race_date': '2024-01-21 00:00:00',
  'description': '2024 F&G Race #2',
  'year': '2024'},
 {'race_id': 240114,
  'race_date': '2024-01-14 00:00:00',
  'description': '2024 F&G Race #1',
  'year': '2024'}]

In [28]:
#race_list = get_races_list(CONN)

races_by_year = {}
for y in years:
    races_for_year = race_list[race_list['year'] == y]
    races_list = races_for_year.to_dict('records')
    races_by_year[y] = races_list

In [29]:
races_metadata = {
    "years": years,
    "races": races_by_year
}

In [30]:
races_metadata

{'years': [2020, 2022, 2023, 2024],
 'races': {2020: [], 2022: [], 2023: [], 2024: []}}

In [ ]:
for year in races_metadata["years"]:
        for race_meta in races_metadata["races"][year]:
            race_results.append(get_race_data(
                'best_time',
                race_meta["race_id"],
                year,
                CONN
            ))

# HERE! Need to clean up this race pull!! Then save in json, and fix up the webpage ...
# including the creation of two tables, one for each discipline

In [22]:
race_data = get_race_data('best_time', '240218', 2024, conn)
race_data[race_data.discipline == 'SNBD']

,bib,name,discipline,team,tier,run1,run2,best_time,points
4,120.0,Hannah Kilmer Choi,SNBD,Will Carter,13.0,25.41,25.34,25.34,8.0
5,101.0,Joel Farber,SNBD,Will Carter,11.0,9998.00,26.47,26.47,8.0
6,114.0,Sacha Fiand,SNBD,Mitch Perreault,12.0,26.52,27.89,26.52,8.0
7,109.0,Mackenzie Hamilton,SNBD,Joanna Perreault,12.0,26.61,29.83,26.61,7.0
8,119.0,Jenna Livingston,SNBD,Stephanie Coward,13.0,27.78,27.40,27.40,7.0
9,121.0,Graham Ramshaw,SNBD,Graham Ramshaw,13.0,28.53,28.58,28.53,6.0
10,107.0,Terence Woodside,SNBD,Mike McTaggart,11.0,28.72,9999.00,28.72,7.0
11,123.0,Bernard Oegema,SNBD,Don French,13.0,28.97,30.17,28.97,5.0
12,116.0,Melanie Leistner Lavoie,SNBD,Mike McTaggart,13.0,29.41,29.00,29.00,4.0
13,102.0,Sarah Ledwidge,SNBD,Joanna Perreault,11.0,30.58,33.57,30.58,6.0


In [23]:
race_data[race_data.discipline == 'SKI']

,bib,name,discipline,team,tier,run1,run2,best_time,points
22,75.0,Nicholas Balan,SKI,Don French,10.0,37.17,37.64,37.17,8.0
23,80.0,Will Stewart,SKI,Adam Grossman,10.0,41.86,37.43,37.43,7.0
24,77.0,Brandon Hune,SKI,Mitch Perreault,10.0,37.55,37.61,37.55,6.0
25,69.0,William Cartar,SKI,Will Carter,9.0,37.65,9998.00,37.65,8.0
26,79.0,Thomas Daly,SKI,Joanna Perreault,10.0,37.67,38.14,37.67,5.0
...,...,...,...,...,...,...,...,...,...
97,40.0,Earl Duffy,SKI,Adam Grossman,5.0,9999.00,9999.00,9999.00,0.0
98,46.0,Craig Pitman,SKI,Adam Grossman,6.0,9999.00,9999.00,9999.00,0.0
99,50.0,Adam Grossman,SKI,Adam Grossman,7.0,9999.00,9999.00,9999.00,0.0
100,66.0,Jeff Cox,SKI,Adam Grossman,9.0,9999.00,9999.00,9999.00,0.0
